# using Microsoft Graph to access Azure resources

“[Build .NET apps with Microsoft Graph](https://learn.microsoft.com/en-us/graph/tutorials/dotnet?tabs=aad)”

## NuGet packages

In [1]:
#r "nuget: Microsoft.Graph"
#r "nuget: Azure.Identity"

Installed Packages Azure.Identity, 1.9.0 Microsoft.Graph, 5.12.0

In [2]:
#r "nuget: SonghayCore"

Installed Packages SonghayCore, 6.0.4

## load conventional `ProgramMetadata`

In [3]:
using System.IO;
using System.Text.Json;
using System.Threading.Tasks;
using Songhay.Models;

var json = File.ReadAllText(Environment.GetEnvironmentVariable("SONGHAY_APP_SETTINGS_PATH"));

JsonDocument.Parse(json).RootElement.TryGetProperty(typeof(ProgramMetadata).Name, out var element)

True

In [4]:
var meta = JsonSerializer.Deserialize<ProgramMetadata>(element);
var restMeta = meta.RestApiMetadataSet["RbacVaultDesktop"];

restMeta.ClaimsSet.Keys

[ app-client-id, app-tenant-id, resource-scope-az-vault, resource-scopes-ms-graph, test-secret-id ]

## define credential-getter for `DeviceCodeCredential`

In [5]:
using Azure.Core;
using Azure.Identity;

public static DeviceCodeCredential GetDeviceCodeCredential(string appClientId, string appTenantId)
{
    var options = new DeviceCodeCredentialOptions
    {
        ClientId = appClientId,
        TenantId = appTenantId,
        DeviceCodeCallback = (info, _) =>
            {
                // Display the device code message to
                // the user. This tells them
                // where to go to sign in and provides the
                // code to use.
                Console.WriteLine(info.Message);
                return Task.FromResult(0);
            }
    };

    var credential = new DeviceCodeCredential(options);

    return credential;
}


## define token-getter for `DeviceCodeCredential`

In [6]:

public static async Task<AccessToken> GetDeviceCodeAccessTokenAsync(DeviceCodeCredential credential, string[] scopes)
{
    var context = new TokenRequestContext(scopes);

    var response = await credential.GetTokenAsync(context);

    return response;
}


## use new `DeviceCodeCredential` for Azure Key Vault

In [7]:
var credential = GetDeviceCodeCredential(
    restMeta.ClaimsSet["app-client-id"],
    restMeta.ClaimsSet["app-tenant-id"]
    );

var response = await GetDeviceCodeAccessTokenAsync(
        credential,
        new [] { restMeta.ClaimsSet["resource-scope-az-vault"] }
    );

!string.IsNullOrWhiteSpace(response.Token)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RDU35APH8 to authenticate.


True

In [8]:
using System.Net.Http;
using System.Net.Http.Headers;
using Songhay.Extensions;

var client = new HttpClient();
var uri = restMeta.ToUri("UriPathTemplateForTestSecret", restMeta.ClaimsSet["test-secret-id"]);

client.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", response.Token);

var result = await client.GetAsync(uri);

await result.Content.ReadAsStringAsync()

{"value":"this-is-the-secret","id":"https://songhay-studio-keyvault.vault.azure.net/secrets/test-secret/467251e536ea488e9a4ab15317f5d808","attributes":{"enabled":true,"created":1685413917,"updated":1685413917,"recoveryLevel":"Recoverable+Purgeable","recoverableDays":90},"tags":{}}

## use new `DeviceCodeCredential` for Microsoft Graph

In [9]:
var credential = GetDeviceCodeCredential(
    restMeta.ClaimsSet["app-client-id"],
    restMeta.ClaimsSet["app-tenant-id"]
    );

var response = await GetDeviceCodeAccessTokenAsync(
        credential,
        new [] { restMeta.ClaimsSet["resource-scopes-ms-graph"] }
    );

!string.IsNullOrWhiteSpace(response.Token)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FKMNEHMUX to authenticate.


True

In [10]:
using Microsoft.Graph;

var client = new GraphServiceClient(credential, new [] { restMeta.ClaimsSet["resource-scopes-ms-graph"] });

var users = await client.Users.GetAsync(config =>
    {
        config.QueryParameters.Select = new[] { "DisplayName", "CompanyName" };
        config.QueryParameters.Top = 5;
        config.QueryParameters.Orderby = new[] { "DisplayName" };
    });

users

Microsoft.Graph.Models.UserCollectionResponse Value index value 0 Microsoft.Graph.Models.User AboutMe <null> AccountEnabled <null> Activities <null> AgeGroup <null> AgreementAcceptances <null> AppRoleAssignments <null> AssignedLicenses <null> AssignedPlans <null> Authentication <null> AuthorizationInfo <null> Birthday <null> BusinessPhones <null> Calendar <null> CalendarGroups <null> Calendars <null> CalendarView <null> Chats <null> City <null> CompanyName <null> ConsentProvidedForMinor <null> ContactFolders <null> Contacts <null> Country <null> CreatedDateTime <null> CreatedObjects <null> CreationType <null> Department <null> DeviceEnrollmentLimit <null> DeviceManagementTroubleshootingEvents <null> DirectReports <null> DisplayName Bryan Wilhite Drive <null> Drives <null> EmployeeHireDate <null> EmployeeId <null> EmployeeLeaveDateTime <null> EmployeeOrgData <null> EmployeeType <null> Events <null> Extensions <null> ExternalUserState <null> ExternalUserStateChangeDateTime <null> FaxNumber <null> FollowedSites <null> GivenName <null> HireDate <null> Identities <null> ImAddresses <null> InferenceClassification <null> Insights <null> Interests <null> IsResourceAccount <null> JobTitle <null> JoinedTeams <null> LastPasswordChangeDateTime <null> LegalAgeGroupClassification <null> LicenseAssignmentStates <null> LicenseDetails <null> Mail <null> MailboxSettings <null> MailFolders <null> MailNickname <null> ManagedAppRegistrations <null> ManagedDevices <null> Manager <null> MemberOf <null> Messages <null> MobilePhone <null> MySite <null> Oauth2PermissionGrants <null> OfficeLocation <null> Onenote <null> OnlineMeetings <null> OnPremisesDistinguishedName <null> OnPremisesDomainName <null> OnPremisesExtensionAttributes <null> OnPremisesImmutableId <null> OnPremisesLastSyncDateTime <null> OnPremisesProvisioningErrors <null> OnPremisesSamAccountName <null> OnPremisesSecurityIdentifier <null> OnPremisesSyncEnabled <null> OnPremisesUserPrincipalName <null> OtherMails <null> Outlook <null> OwnedDevices <null> OwnedObjects <null> PasswordPolicies <null> PasswordProfile <null> PastProjects <null> People <null> Photo <null> Photos <null> Planner <null> PostalCode <null> PreferredDataLocation <null> PreferredLanguage <null> PreferredName <null> Presence <null> ProvisionedPlans <null> ProxyAddresses <null> RegisteredDevices <null> Responsibilities <null> Schools <null> ScopedRoleMemberOf <null> SecurityIdentifier <null> Settings <null> ShowInAddressList <null> SignInActivity <null> SignInSessionsValidFromDateTime <null> Skills <null> State <null> StreetAddress <null> Surname <null> Teamwork <null> Todo <null> TransitiveMemberOf <null> UsageLocation <null> UserPrincipalName <null> UserType <null> DeletedDateTime <null> AdditionalData (empty) BackingStore Microsoft.Kiota.Abstractions.Store.InMemoryBackingStore ReturnOnlyChangedValues False InitializationCompleted True Id <null> OdataType #microsoft.graph.user AdditionalData key value @odata.context https://graph.microsoft.com/v1.0/$metadata#users(displayName,companyName) BackingStore Microsoft.Kiota.Abstractions.Store.InMemoryBackingStore ReturnOnlyChangedValues False InitializationCompleted True OdataCount <null> OdataNextLink <null>

In [11]:
var me = await client.Me.GetAsync();

me

AboutMe,<null>
AccountEnabled,<null>
Activities,<null>
AgeGroup,<null>
AgreementAcceptances,<null>
AppRoleAssignments,<null>
AssignedLicenses,<null>
AssignedPlans,<null>
Authentication,<null>
AuthorizationInfo,<null>
Birthday,<null>


## use new `DeviceCodeCredential` for Azure Storage

In [ ]:
#r "nuget: Azure.Storage.Blobs"


In [ ]:
using Azure.Storage.Blobs;
using Azure.Storage.Blobs.Models;

var uri = new Uri("https://fubar.blob.core.windows.net", UriKind.Absolute);
var client = new BlobServiceClient(uri, credential);

var containerClient = client.GetBlobContainerClient("foo");

var blobClient = containerClient.GetBlobClient("kb-1064618963.json");

blobClient.DownloadContent()